In [1]:
import numpy as np
import pandas as pd 
import scipy.sparse

In [2]:

train =pd.read_csv(r"data\sales_train_v2.csv")

In [3]:
train["date"] = pd.to_datetime(train.date,format="%d.%m.%Y")

In [4]:
train["rev"] = train["item_cnt_day"]
train2 = train.drop(columns=["item_price","item_cnt_day"])

In [5]:
grouped = train2.groupby(["date","date_block_num","shop_id","item_id"])
train2 = grouped.agg(sum).reset_index()

In [6]:
train2.loc[(train2["item_id"]==2573) & (train2["shop_id"]==25),:]

,date,date_block_num,shop_id,item_id,rev
10526,2013-01-03,0,25,2573,3.0
177151,2013-02-17,1,25,2573,1.0
181411,2013-02-18,1,25,2573,1.0
238982,2013-03-04,2,25,2573,1.0
404685,2013-04-20,3,25,2573,1.0
506841,2013-05-24,4,25,2573,1.0


In [7]:
import datetime
from datetime import date
from datetime import timedelta

In [8]:

#train2["WoY"]=train2["date"].apply(lambda x : x.strftime("%V"))
#train2["DoW"]=train2["date"].apply(lambda x: x.weekday())
#train2["day"]=train2["date"].apply(lambda x: x.day)
train2["month"]=train2["date"].apply(lambda x: x.month)
train2["year"]=train2["date"].apply(lambda x: x.year)
train2["mon"]=train2["date_block_num"]
train2.head()

,date,date_block_num,shop_id,item_id,rev,month,year,mon
0,2013-01-01,0,2,991,1.0,1,2013,0
1,2013-01-01,0,2,1472,1.0,1,2013,0
2,2013-01-01,0,2,1905,1.0,1,2013,0
3,2013-01-01,0,2,2920,2.0,1,2013,0
4,2013-01-01,0,2,3320,1.0,1,2013,0


In [9]:
items = pd.read_csv("data/items.csv")
train2 = pd.merge(train2,items.loc[:,["item_id","item_category_id"]],on="item_id",how="inner")

In [10]:
train2.head()

,date,date_block_num,shop_id,item_id,rev,month,year,mon,item_category_id
0,2013-01-01,0,2,991,1.0,1,2013,0,67
1,2013-01-02,0,53,991,1.0,1,2013,0,67
2,2013-01-03,0,26,991,1.0,1,2013,0,67
3,2013-01-03,0,42,991,1.0,1,2013,0,67
4,2013-01-05,0,16,991,1.0,1,2013,0,67


In [11]:
def create_month_agg(data, groupbyfeats):
    ## count column
    val = ["rev"]
    
    days_in_month = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    def get_divisor(mon):
        return days_in_month[((mon+12)%12) + 1]
    
    def clip(x):
        if(x>20):
            return 20
        if(x<0):
            return 0
        return x
    
    cur_feats = groupbyfeats.copy()
    cur_feats.append("mon")
    print(cur_feats)
    temp = data.loc[:,cur_feats+val]
    grouped = temp.groupby(cur_feats)
    temp = grouped.agg([sum,'count']).reset_index()
    temp.columns= cur_feats+['sum','count']
    rev_field = "rev_per_day"
    temp[rev_field] =  temp["sum"] /(temp["mon"].apply(lambda x : get_divisor(x)))
    temp[rev_field] = temp[rev_field].apply(lambda x: np.log1p(x) if(x>0) else 0)

    new_fields = cur_feats.copy()
    new_fields.append(rev_field)
    flookup = temp.loc[:,new_fields]
    #flookup.rename(columns={"mon":"sale_month"}, inplace=True)
    return flookup

In [12]:
def generate_monthly_features(data, flookup, f1):
    temp = data
    mon_agg = flookup.groupby("mon")["rev_per_day"].agg('mean').reset_index().rename(columns={"mon":"mean_mon","rev_per_day":"mean_rev"})
    #print(mon_agg.columns)
    nmean = flookup.rev_per_day.mean()
    print(nmean)
    for i in range(1,13):
        fname = "mon_"+str(i)
        
        temp[fname]=temp["sale_month"].apply(lambda x : max(0,x-12))
        fleft = f1.copy()
        fleft.append(fname)
        fright = f1.copy()
        fright.append("mon")    
        ofname = ("_".join(fleft))+"_rev_day_mon_"+str(i)
        
        temp = pd.merge(temp,flookup,left_on=fleft, right_on=fright,how="left")
        temp2 = pd.merge(temp,mon_agg,left_on=fname,right_on="mean_mon",how="left")
        #print(temp.head(2))
        #print("%s - null count = %f"%(ofname,temp.rev_per_day.isnull().sum()))
        temp.rev_per_day.fillna(temp2["mean_rev"],inplace=True)
        #print(temp.head(2))
        #print("%s - null count = %f"%(ofname,temp.rev_per_day.isnull().sum()))
        temp.rev_per_day.fillna(nmean,inplace=True)
        #print(temp.head(2))
        print("%s - null count = %f"%(ofname,temp.rev_per_day.isnull().sum()))
        #print(temp.columns)
        temp.drop(columns=[fname, "mon"],inplace=True,axis=1)
        temp.rename(columns={"rev_per_day":ofname},inplace=True)
    return temp

In [13]:
def generate_features(base_data):
    ## Features for item_id, shop_id
    f1 = ["item_id","shop_id"]
    lookup_data=create_month_agg(train2,f1)
    base_data=generate_monthly_features(base_data,lookup_data,f1)
    
    ## Features for item_id
    f2=["item_id"]
    lookup_data=create_month_agg(train2,f2)    
    base_data=generate_monthly_features(base_data,lookup_data,f2)
    #print(base_data.head())
    
    ## Features for shop_id
    f3=["shop_id"]
    lookup_data=create_month_agg(train2,f3)
    base_data=generate_monthly_features(base_data,lookup_data,f3)
    #print(base_data.head())
    
    f1 = ["item_category_id","shop_id"]
    lookup_data=create_month_agg(train2,f1)
    base_data=generate_monthly_features(base_data,lookup_data,f1)
    #print(base_data.head())
    
    f1 = ["item_category_id"]
    lookup_data=create_month_agg(train2,f1)
    base_data=generate_monthly_features(base_data,lookup_data,f1)
    #print(base_data.head())
    
    return base_data


In [14]:

f2 = ["item_id","shop_id","year", "item_category_id"]
train_data = create_month_agg(train2, f2)
train_data.rename(columns={"mon":"sale_month","rev_per_day":"true_rev_per_day"},inplace=True)
#train_data.head()

train_data=generate_features(train_data)
train_data.head()

['item_id', 'shop_id', 'year', 'item_category_id', 'mon']
['item_id', 'shop_id', 'mon']
0.0644514326947
item_id_shop_id_mon_1_rev_day_mon_1 - null count = 0.000000
item_id_shop_id_mon_2_rev_day_mon_2 - null count = 0.000000
item_id_shop_id_mon_3_rev_day_mon_3 - null count = 0.000000
item_id_shop_id_mon_4_rev_day_mon_4 - null count = 0.000000
item_id_shop_id_mon_5_rev_day_mon_5 - null count = 0.000000
item_id_shop_id_mon_6_rev_day_mon_6 - null count = 0.000000
item_id_shop_id_mon_7_rev_day_mon_7 - null count = 0.000000
item_id_shop_id_mon_8_rev_day_mon_8 - null count = 0.000000
item_id_shop_id_mon_9_rev_day_mon_9 - null count = 0.000000
item_id_shop_id_mon_10_rev_day_mon_10 - null count = 0.000000
item_id_shop_id_mon_11_rev_day_mon_11 - null count = 0.000000
item_id_shop_id_mon_12_rev_day_mon_12 - null count = 0.000000
['item_id', 'mon']
0.267235547598
item_id_mon_1_rev_day_mon_1 - null count = 0.000000
item_id_mon_2_rev_day_mon_2 - null count = 0.000000
item_id_mon_3_rev_day_mon_3 - nu

,item_id,shop_id,year,item_category_id,sale_month,true_rev_per_day,item_id_shop_id_mon_1_rev_day_mon_1,item_id_shop_id_mon_2_rev_day_mon_2,item_id_shop_id_mon_3_rev_day_mon_3,item_id_shop_id_mon_4_rev_day_mon_4,...,item_category_id_mon_3_rev_day_mon_3,item_category_id_mon_4_rev_day_mon_4,item_category_id_mon_5_rev_day_mon_5,item_category_id_mon_6_rev_day_mon_6,item_category_id_mon_7_rev_day_mon_7,item_category_id_mon_8_rev_day_mon_8,item_category_id_mon_9_rev_day_mon_9,item_category_id_mon_10_rev_day_mon_10,item_category_id_mon_11_rev_day_mon_11,item_category_id_mon_12_rev_day_mon_12
0,0,54,2014,40,20,0.032790,0.069887,0.069887,0.069887,0.069887,...,6.658353,6.658353,6.658353,6.658353,6.658353,6.658353,6.658353,6.658353,6.658353,6.658353
1,1,55,2014,76,15,0.064539,0.059981,0.059981,0.059981,0.059981,...,2.987761,2.987761,2.987761,2.987761,2.987761,2.987761,2.987761,2.987761,2.987761,2.987761
2,1,55,2014,76,18,0.031749,0.057643,0.057643,0.057643,0.057643,...,1.441210,1.441210,1.441210,1.441210,1.441210,1.441210,1.441210,1.441210,1.441210,1.441210
3,1,55,2014,76,19,0.031749,0.061366,0.061366,0.061366,0.061366,...,1.478668,1.478668,1.478668,1.478668,1.478668,1.478668,1.478668,1.478668,1.478668,1.478668
4,1,55,2014,76,20,0.032790,0.069887,0.069887,0.069887,0.069887,...,1.616082,1.616082,1.616082,1.616082,1.616082,1.616082,1.616082,1.616082,1.616082,1.616082


In [15]:
import pickle

pickle.dump(train_data, open("train_data_with_feats.p","wb"))
pickle.dump(train2, open("train2.p","wb"))

In [16]:
from sklearn import datasets, linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [17]:
val_set_rule = (train_data["sale_month"]==33)
train_set_rule = ((train_data["sale_month"]!=33)) # & (train_data["sale_month"]>2))
xtrain = (train_data.loc[train_set_rule,:]).iloc[:,6:]
#print(xtrain.head())
ytrain = (train_data.loc[train_set_rule,:]).iloc[:,5]
#print(ytrain.head())

xval = (train_data.loc[val_set_rule,:]).iloc[:,6:]
#print(xval.head())
yval = (train_data.loc[val_set_rule,:]).iloc[:,5]
#print(yval.head())


In [23]:
# Create linear regression object
regr = DecisionTreeRegressor(max_depth=15)

# Train the model using the training sets
regr.fit(xtrain, ytrain)

DecisionTreeRegressor(criterion='mse', max_depth=15, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [24]:
print("training MSE: {%f}" % mean_squared_error(ytrain,regr.predict(xtrain)))
print("validation MSE: {%f}" % mean_squared_error(yval,regr.predict(xval)))

training MSE: {0.005796}
validation MSE: {0.008593}


## Original
training MSE: {0.032447}
validation MSE: {0.078754}

## missing + log
training MSE: {0.005783}
validation MSE: {0.010810}

In [25]:
cols = train_data.columns
f_imp = dict()
for i in range(len(regr.feature_importances_)):
    f_imp[cols[i+6]]=regr.feature_importances_[i]

for k in f_imp.keys():
    if(f_imp[k]>0.05):
        print("%s -> %f"%(k,f_imp[k]))

item_id_shop_id_mon_1_rev_day_mon_1 -> 0.077023
item_id_mon_1_rev_day_mon_1 -> 0.224256
item_category_id_mon_5_rev_day_mon_5 -> 0.128502
item_category_id_mon_12_rev_day_mon_12 -> 0.066308


In [ ]:
from xgboost import XGBClassifier
regr_2 = XGBClassifier()
#regr_2.fit(xtrain, ytrain)

print("training MSE: {%f}" % mean_squared_error(ytrain,regr_2.predict(xtrain)))
print("validation MSE: {%f}" % mean_squared_error(yval,regr_2.predict(xval)))

In [26]:
test = pd.read_csv(r"data\test.csv")
test = pd.merge(test,items.loc[:,["item_id","item_category_id"]],on="item_id",how="left")
test.fillna(0)
test["year"]=2015
test["sale_month"]=34
test["true_rev_per_day"]=0.0

test.head()

,ID,shop_id,item_id,item_category_id,year,sale_month,true_rev_per_day
0,0,5,5037,19,2015,34,0.0
1,1,5,5320,55,2015,34,0.0
2,2,5,5233,19,2015,34,0.0
3,3,5,5232,23,2015,34,0.0
4,4,5,5268,20,2015,34,0.0


In [27]:

test_data=generate_features(test)
test_data.head()

['item_id', 'shop_id', 'mon']
0.0644514326947
item_id_shop_id_mon_1_rev_day_mon_1 - null count = 0.000000
item_id_shop_id_mon_2_rev_day_mon_2 - null count = 0.000000
item_id_shop_id_mon_3_rev_day_mon_3 - null count = 0.000000
item_id_shop_id_mon_4_rev_day_mon_4 - null count = 0.000000
item_id_shop_id_mon_5_rev_day_mon_5 - null count = 0.000000
item_id_shop_id_mon_6_rev_day_mon_6 - null count = 0.000000
item_id_shop_id_mon_7_rev_day_mon_7 - null count = 0.000000
item_id_shop_id_mon_8_rev_day_mon_8 - null count = 0.000000
item_id_shop_id_mon_9_rev_day_mon_9 - null count = 0.000000
item_id_shop_id_mon_10_rev_day_mon_10 - null count = 0.000000
item_id_shop_id_mon_11_rev_day_mon_11 - null count = 0.000000
item_id_shop_id_mon_12_rev_day_mon_12 - null count = 0.000000
['item_id', 'mon']
0.267235547598
item_id_mon_1_rev_day_mon_1 - null count = 0.000000
item_id_mon_2_rev_day_mon_2 - null count = 0.000000
item_id_mon_3_rev_day_mon_3 - null count = 0.000000
item_id_mon_4_rev_day_mon_4 - null cou

,ID,shop_id,item_id,item_category_id,year,sale_month,true_rev_per_day,item_id_shop_id_mon_1_rev_day_mon_1,item_id_shop_id_mon_2_rev_day_mon_2,item_id_shop_id_mon_3_rev_day_mon_3,...,item_category_id_mon_3_rev_day_mon_3,item_category_id_mon_4_rev_day_mon_4,item_category_id_mon_5_rev_day_mon_5,item_category_id_mon_6_rev_day_mon_6,item_category_id_mon_7_rev_day_mon_7,item_category_id_mon_8_rev_day_mon_8,item_category_id_mon_9_rev_day_mon_9,item_category_id_mon_10_rev_day_mon_10,item_category_id_mon_11_rev_day_mon_11,item_category_id_mon_12_rev_day_mon_12
0,0,5,5037,19,2015,34,0.0,0.03279,0.03279,0.03279,...,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284
1,1,5,5320,55,2015,34,0.0,0.07058,0.07058,0.07058,...,5.792912,5.792912,5.792912,5.792912,5.792912,5.792912,5.792912,5.792912,5.792912,5.792912
2,2,5,5233,19,2015,34,0.0,0.07058,0.07058,0.07058,...,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284
3,3,5,5232,23,2015,34,0.0,0.07058,0.07058,0.07058,...,5.175208,5.175208,5.175208,5.175208,5.175208,5.175208,5.175208,5.175208,5.175208,5.175208
4,4,5,5268,20,2015,34,0.0,0.07058,0.07058,0.07058,...,6.060991,6.060991,6.060991,6.060991,6.060991,6.060991,6.060991,6.060991,6.060991,6.060991


In [28]:
pickle.dump(test_data, open("test_data_with_feats.p","wb"))

In [29]:
test_data["pred"] = regr.predict(test_data.iloc[:,7:])

test_data["item_cnt_month"]= np.clip(test_data["pred"],0,20)
test_data.head()

,ID,shop_id,item_id,item_category_id,year,sale_month,true_rev_per_day,item_id_shop_id_mon_1_rev_day_mon_1,item_id_shop_id_mon_2_rev_day_mon_2,item_id_shop_id_mon_3_rev_day_mon_3,...,item_category_id_mon_5_rev_day_mon_5,item_category_id_mon_6_rev_day_mon_6,item_category_id_mon_7_rev_day_mon_7,item_category_id_mon_8_rev_day_mon_8,item_category_id_mon_9_rev_day_mon_9,item_category_id_mon_10_rev_day_mon_10,item_category_id_mon_11_rev_day_mon_11,item_category_id_mon_12_rev_day_mon_12,pred,item_cnt_month
0,0,5,5037,19,2015,34,0.0,0.03279,0.03279,0.03279,...,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,0.043429,0.043429
1,1,5,5320,55,2015,34,0.0,0.07058,0.07058,0.07058,...,5.792912,5.792912,5.792912,5.792912,5.792912,5.792912,5.792912,5.792912,0.045467,0.045467
2,2,5,5233,19,2015,34,0.0,0.07058,0.07058,0.07058,...,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,5.325284,0.046055,0.046055
3,3,5,5232,23,2015,34,0.0,0.07058,0.07058,0.07058,...,5.175208,5.175208,5.175208,5.175208,5.175208,5.175208,5.175208,5.175208,0.066960,0.066960
4,4,5,5268,20,2015,34,0.0,0.07058,0.07058,0.07058,...,6.060991,6.060991,6.060991,6.060991,6.060991,6.060991,6.060991,6.060991,0.045467,0.045467


In [30]:
sub = test_data.loc[:,["ID","item_cnt_month"]]
sub.to_csv("data/sub_tree.csv",index=False)

In [35]:
test_data["pred2"] = regr_2.predict(test_data.iloc[:,7:-2])

test_data["item_cnt_month2"]= np.clip(test_data["pred"],0,20)
test_data.head()


,ID,shop_id,item_id,item_category_id,year,sale_month,true_rev_per_day,rev_day_mon_1,rev_day_mon_2,rev_day_mon_3,...,rev_day_mon_5,rev_day_mon_6,rev_day_mon_7,rev_day_mon_8,rev_day_mon_9,rev_day_mon_10,rev_day_mon_11,rev_day_mon_12,pred,item_cnt_month
0,0,5,5037,19,2016,34,0.0,0.033333,0.033333,0.033333,...,204.466667,204.466667,204.466667,204.466667,204.466667,204.466667,204.466667,204.466667,0.084722,2.541647
1,1,5,5320,55,2016,34,0.0,0.000000,0.000000,0.000000,...,326.966667,326.966667,326.966667,326.966667,326.966667,326.966667,326.966667,326.966667,0.084722,2.541647
2,2,5,5233,19,2016,34,0.0,0.000000,0.000000,0.000000,...,204.466667,204.466667,204.466667,204.466667,204.466667,204.466667,204.466667,204.466667,0.084722,2.541647
3,3,5,5232,23,2016,34,0.0,0.000000,0.000000,0.000000,...,175.833333,175.833333,175.833333,175.833333,175.833333,175.833333,175.833333,175.833333,0.084113,2.523393
4,4,5,5268,20,2016,34,0.0,0.000000,0.000000,0.000000,...,427.800000,427.800000,427.800000,427.800000,427.800000,427.800000,427.800000,427.800000,0.084113,2.523393


In [36]:
sub = test_data.loc[:,["ID","item_cnt_month"]]
sub.to_csv("data/sub_adaboost.csv",index=False)